<a href="https://www.kaggle.com/code/aniruddhapa/enzyme-stability-xgboost-baseline-model-0-14?scriptVersionId=161650282" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# "Predicting Enzyme Thermostability: A Biotechnological Challenge by Novozymes"

The Kaggle competition focuses on predicting the thermostability of enzyme variants, which is crucial in biotechnology for optimizing enzyme performance in various applications. Novozymes, a leading biotech company, seeks computational models to enhance protein stability prediction. Enzymes play vital roles in industries, from detergent manufacturing to biofuel production. The challenge involves developing machine learning models using experimental melting temperature data, and participants are encouraged to contribute to advancements in protein stability prediction. The goal is to improve the design of proteins, such as enzymes and therapeutics, for enhanced stability, rapid development, and lower costs. The competition offers access to high-throughput screening lab data and aims to address fundamental challenges in predicting protein thermal stability. Participants are invited to contribute to the biotech industry's progress and align with Novozymes' commitment to environmental and social impact.

## Here's a breakdown of the key steps : 

### Data Loading and Preprocessing:

* Loaded the training and test datasets.
* Incorporated train updates into the training dataset.
* Handled missing values and updated pH and tm values based on train updates.

### Exploratory Data Analysis (EDA):

* Utilized Plotly to create interactive visualizations.
* Plotted a histogram for the target column tm to show the distribution.
* Analyzed the length of protein sequences in both training and test datasets using histograms.
* Checked the statistics for protein sequence lengths.

### 3D Scatter Plot for Protein Structure Visualization:

* Used Plotly to create a 3D scatter plot for protein structures.
* Highlighted different elements with distinct colors.

### Biopandas for PDB File Analysis:

* Utilized Biopandas to read and analyze PDB files.
* Extracted datasets for ATOM, HETATM, ANISOU, and OTHERS.

### Label Encoding for Sequences:

* Transformed protein sequences into numerical format using Label Encoding.
* Prepared the data for model training.

### Model Training with XGBoost:

* Split the dataset into training and testing sets.
* Used XGBoost, a gradient boosting library, to create a regression model.
* Evaluated the model's performance using the root mean squared error (RMSE).

### Model Prediction on Test Data:

* Processed the test dataset in a similar manner to the training data.
* Used the trained XGBoost model to predict thermostability (tm) values for the test dataset.

### Submission:

* Created a submission file containing predicted thermostability values for the test data.
* Saved the submission file as 'submission.csv'.



# Introduction:

Enzymes play a crucial role as catalysts in biological reactions, and predicting their thermostability is vital for advancing biotechnology applications. In the Kaggle competition titled "Novozymes Enzyme Stability Prediction," participants are challenged to develop a model that predicts the thermostability of enzyme variants. This prediction is based on experimental melting temperature data obtained from Novozymes's high-throughput screening lab.

The goal of the competition is to address the fundamental problem of improving protein stability, a key factor in enzyme engineering for diverse applications. In this Kaggle notebook, we present a comprehensive solution to this challenge, covering data exploration, preprocessing, visualization, and model training using the powerful XGBoost algorithm. Our approach not only predicts enzyme thermostability but also provides insights into protein sequence lengths, 3D structure visualization, and analysis of PDB (Protein Data Bank) files.

# Data Loading and Preprocessing:

In [1]:
#Displaying complete column values

# pd.set_option('display.max_colwidth', 0)

In [2]:
#Reset all options staring with display
# pd.reset_option('^display.', silent=True)


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_profiling
import time
import torch
import torch.nn as nn

In [4]:
class paths:
    TRAIN = "../input/novozymes-enzyme-stability-prediction/train.csv"
    TRAIN_UPDATES='../input/novozymes-enzyme-stability-prediction/train_updates_20220929.csv'
    TEST = "/kaggle/input/novozymes-enzyme-stability-prediction/test.csv"
    SUBMISSION = "/kaggle/input/novozymes-enzyme-stability-prediction/sample_submission.csv"
    PDB_FILE = "/kaggle/input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"

In [5]:
#Creating Train dataset incorporating train updates
train_df=pd.read_csv(paths.TRAIN)
train_df_updates=pd.read_csv(paths.TRAIN_UPDATES,index_col='seq_id')
all_features_nan=train_df_updates.isnull().all('columns')
drop_indices=train_df_updates[all_features_nan].index
train_df=train_df.drop(index=drop_indices)
swap_ph_tm_indices=train_df_updates[~all_features_nan].index
train_df.loc[swap_ph_tm_indices,['pH','tm']]=train_df_updates.loc[swap_ph_tm_indices,['pH','tm']]

#Test data 
test_df=pd.read_csv(paths.TEST)

#Displaying Train and Test dataset
print(f"Train Dataframe has shape:{train_df.shape}")
print(f"Test Dataframe has shape:{test_df.shape}")
display(train_df.head())
display(test_df.head())

Train Dataframe has shape:(28981, 5)
Test Dataframe has shape:(2413, 4)


,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


# Exploratory Data Analysis (EDA):



In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig=go.Figure()
colors=["#FFD43B",'#002EFF']

i=0

fig.add_trace(
    go.Histogram(x=train_df['tm'],
                 name='tm',
                 hovertemplate='target column'+'%{y::2f}',
                 marker=dict(color=colors[i])
                )
)
fig.update_xaxes(
        title_text='Target column',
        title_font_color=colors[i],
        tickfont_color=colors[i])
fig.update_yaxes(
        title_text='Frequency',
        title_font_color=colors[i],
        tickfont_color=colors[i])

fig.update_layout(height=800,
                  width=800,
                  title_text='Target Column: Higher tm means the protien variant is more stable',
                  template="plotly_dark",
                  xaxis=dict(color="#FF9300"),
                  yaxis=dict(color="#FF9300"))

fig.update_traces(marker_line_width=0.1,marker_line_color=colors[1])
fig.show()
            
    
        

In [7]:
train_df['protein_sequence_len']=train_df['protein_sequence'].apply(lambda x:len(x))
test_df['protein_sequence_len']=test_df['protein_sequence'].apply(lambda x:len(x))
print('---------TRAIN--------')
display(train_df[['protein_sequence_len']].describe())
print('---------TEST--------')
display(test_df[['protein_sequence_len']].describe())

---------TRAIN--------


,protein_sequence_len
count,28981.000000
mean,450.468617
std,415.159049
min,5.000000
25%,212.000000
50%,351.000000
75%,537.000000
max,8798.000000


---------TEST--------


,protein_sequence_len
count,2413.000000
mean,220.968090
std,0.175798
min,220.000000
25%,221.000000
50%,221.000000
75%,221.000000
max,221.000000


### Sequence Length

Sequence length seems to vary for *train.csv* while it is constant for *test.csv*

train data has a heavy outlier of length 8798. The distribution is highly right skewed.

# 3D Scatter Plot for Protein Structure Visualization:

In [8]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = go.Figure()
colors = ["#FF00F0", "#BD00FF"]

i = 0
# Add one subplot
fig.add_trace(
    go.Histogram(x=train_df["protein_sequence_len"],
               name="protein_sequence_len",
               hovertemplate = 'Protein Sequence Length:  %{x:.2f}' + '<br>Frequency: %{y:.2f}</br>',
               marker=dict(color=colors[i])
              )
)
fig.update_xaxes(
    title_text="Protein Sequence Length",
    title_font_color=colors[i],
    tickfont_color=colors[i],
    range=[0, 1500]
) 
fig.update_yaxes(
    title_text = "Frequency", 
    title_font_color=colors[i], 
    tickfont_color=colors[i]
) 

fig.update_layout(height=800,
                  width=1000,
                  title_text="Potein Sequence Length Histogram.",
                  template="plotly_dark",
                  xaxis=dict(color="#FF9300"),
                  yaxis=dict(color="#FF9300"))
fig.update_traces(marker_line_width=2,marker_line_color=colors[1])

fig.show()

# Biopandas for PDB File Analysis:

# What is PDB File Format

**The Protein Data Bank (pdb) file format is a textual file format describing the three-dimensional structures of molecules held in the Protein Data Bank. The pdb format accordingly provides for description and annotation of protein and nucleic acid structures including atomic coordinates, secondary structure assignments, as well as atomic connectivity.**


**The Protein Data Bank (PDB) is a database for the three-dimensional structural data of large biological molecules, such as proteins and nucleic acids.**

In [9]:
# Biopandas is a python package for working with molecular structures in pandas DataFrames.

!pip install biopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 10.9 MB/s eta 0:00:00


In [10]:
from biopandas.pdb import PandasPdb

pdb_df =  PandasPdb().read_pdb(paths.PDB_FILE)
pdb_df.df.keys()

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])

In [11]:
def sep():
    print('-'*100)

In [12]:
atom_df=pdb_df.df['ATOM']
hetatm_df=pdb_df.df['HETATM']
anisou_df=pdb_df.df['ANISOU']
others_df=pdb_df.df['OTHERS']
print(f"ATOM dataset is of shape: {atom_df.shape}"), sep()
print(f"HETATM dataset is of shape: {hetatm_df.shape}"), sep()
print(f"ANISOU dataset is of shape: {anisou_df.shape}"), sep()
print(f"OTHERS dataset is of shape: {others_df.shape}"), sep()

display(atom_df.head())
display(hetatm_df.head())
display(anisou_df.head())
display(others_df.head())

ATOM dataset is of shape: (3317, 21)
----------------------------------------------------------------------------------------------------
HETATM dataset is of shape: (0, 21)
----------------------------------------------------------------------------------------------------
ANISOU dataset is of shape: (0, 21)
----------------------------------------------------------------------------------------------------
OTHERS dataset is of shape: (2, 3)
----------------------------------------------------------------------------------------------------


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,VAL,,A,1,,...,34.064,-6.456,50.464,1.0,45.11,,,N,NaN,0
1,ATOM,2,,H,,VAL,,A,1,,...,33.576,-6.009,51.228,1.0,45.11,,,H,NaN,1
2,ATOM,3,,H2,,VAL,,A,1,,...,33.882,-7.449,50.477,1.0,45.11,,,H,NaN,2
3,ATOM,4,,H3,,VAL,,A,1,,...,35.060,-6.323,50.566,1.0,45.11,,,H,NaN,3
4,ATOM,5,,CA,,VAL,,A,1,,...,33.643,-5.877,49.162,1.0,45.11,,,C,NaN,4


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,"U(1,1)","U(2,2)","U(3,3)","U(1,2)","U(1,3)","U(2,3)",blank_4,element_symbol,charge,line_idx


,record_name,entry,line_idx
0,TER,3318 LYS A 221,3317
1,END,,3318


In [13]:
atom_df.columns

Index(['record_name', 'atom_number', 'blank_1', 'atom_name', 'alt_loc',
       'residue_name', 'blank_2', 'chain_id', 'residue_number', 'insertion',
       'blank_3', 'x_coord', 'y_coord', 'z_coord', 'occupancy', 'b_factor',
       'blank_4', 'segment_id', 'element_symbol', 'charge', 'line_idx'],
      dtype='object')

In [14]:
import plotly.express as px

fig = px.scatter_3d(atom_df, x = "x_coord",
                    y = "y_coord",
                    z = "z_coord",
                    color = "element_symbol",
                    color_discrete_sequence = ["#84FFA9", "#00FFF7", "#003AFF", "#F000FF", "#FBFF00"])
fig.update_traces(marker = dict(size = 3))
fig.update_coloraxes(showscale = False)
fig.update_layout(template = "plotly_dark")
fig.show()

In [15]:
from scipy.sparse import csr_matrix
train_df=train_df[train_df['protein_sequence_len']<=221]
train_df.reset_index(inplace=True)

In [16]:
train_df.isnull().sum()

index                     0
seq_id                    0
protein_sequence          0
pH                        1
data_source             766
tm                        0
protein_sequence_len      0
dtype: int64

# Label Encoding for Sequences:

In [17]:
sequences=[list(string) for string in train_df['protein_sequence'].values.tolist()]
sequences_train=pd.DataFrame(sequences)
sequences_train.head()

,0,1,2,3,4,5,6,7,8,9,...,211,212,213,214,215,216,217,218,219,220
0,A,A,F,Q,V,T,S,N,E,I,...,None,None,None,None,None,None,None,None,None,None
1,A,A,G,G,Q,P,Q,G,A,T,...,A,Q,Q,Q,C,N,None,None,None,None
2,A,A,I,G,I,G,I,L,G,G,...,None,None,None,None,None,None,None,None,None,None
3,A,A,K,S,G,D,A,E,E,A,...,None,None,None,None,None,None,None,None,None,None
4,A,A,L,A,L,G,L,P,A,F,...,None,None,None,None,None,None,None,None,None,None


In [18]:
from sklearn.preprocessing import LabelEncoder

sequences_train = sequences_train.apply(LabelEncoder().fit_transform)
sequences_train["tm"] = train_df["tm"]
sequences_train.head()

,0,1,2,3,4,5,6,7,8,9,...,212,213,214,215,216,217,218,219,220,tm
0,0,0,4,13,17,16,15,11,3,7,...,20,19,20,20,20,20,20,20,18,49.7
1,0,0,5,5,13,12,13,5,0,16,...,13,13,13,1,11,20,20,20,18,45.1
2,0,0,7,5,7,5,7,9,5,5,...,20,19,20,20,20,20,20,20,18,62.8
3,0,0,8,15,5,2,0,3,3,0,...,20,19,20,20,20,20,20,20,18,36.3
4,0,0,9,0,9,5,9,12,0,4,...,20,19,20,20,20,20,20,20,18,83.0


In [19]:
sequences_train.isnull().sum()

0      0
1      0
2      0
3      0
4      0
      ..
217    0
218    0
219    0
220    0
tm     0
Length: 222, dtype: int64

# Model Training with XGBoost:

In [20]:
from sklearn.model_selection import train_test_split
import xgboost

X = sequences_train.loc[:, sequences_train.columns != "tm"]
y = sequences_train.loc[:, sequences_train.columns == "tm"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# create an xgboost regression model
model = xgboost.XGBRegressor(n_estimators=15, max_depth=7)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [21]:
model

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=15, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

# Model Prediction on Test Data:

In [22]:
from sklearn.metrics import mean_squared_error as MSE

rmse=np.sqrt(MSE(y_test,y_pred))
print(rmse)

12.553033431321708


In [23]:
y_pred[:5]

array([44.704197, 56.43628 , 50.510666, 56.202457, 65.04355 ],
      dtype=float32)

In [24]:
from scipy import stats

stats.spearmanr(y_test, y_pred)

SpearmanrResult(correlation=0.2958411727248812, pvalue=1.1096646549822055e-32)

In [25]:
from scipy.sparse import csr_matrix

test_df = test_df[test_df["protein_sequence_len"]<=221]
sequences = [list(string) for string in test_df["protein_sequence"].values.tolist()]
sequences_test = pd.DataFrame(sequences)
sequences_test = sequences_test.apply(LabelEncoder().fit_transform)
sequences_test.head()

,0,1,2,3,4,5,6,7,8,9,...,211,212,213,214,215,216,217,218,219,220
0,0,0,0,0,0,0,0,0,0,0,...,7,11,7,5,1,8,13,15,2,6
1,0,0,0,0,0,0,0,0,0,0,...,7,11,7,5,1,8,13,15,2,6
2,0,0,0,0,0,0,0,0,0,0,...,10,11,6,2,5,16,11,4,4,13
3,0,0,0,0,0,0,0,0,0,0,...,7,11,7,5,1,8,13,15,2,6
4,0,0,0,0,0,0,0,0,0,0,...,7,11,7,5,1,8,13,15,2,6


# Submission:

In [26]:
submission =pd.DataFrame()
submission['tm']=model.predict(sequences_test)
submission['seq_id']=test_df['seq_id']
submission.to_csv('submission.csv',index=False)